In [73]:
import pandas as pd, numpy as np

# Load the 2016 data set into a pandas DataFrame
df2016 = pd.read_csv("/content/drive/MyDrive/CIND 820 Capstone Project/2016_Mississauga_Business_Directory.csv")

#NAICSTitle and NAICSDescr are in the wrong columns from 2016 to 2018.  2019 and 2021 are correct.
#Rename NAICSTitle to NAICSDescr ; NAICSSecto to NAICSTitle
df2016.rename(columns = {'NAICSDescr':'NAICSCat', 'NAICSTitle':'NAICSDescr'}, inplace = True)

# Create a column to hold the year
df2016['Year'] = 2016

# Load the 2017 data set into a pandas DataFrame
df2017 = pd.read_csv("/content/drive/MyDrive/CIND 820 Capstone Project/2017_Mississauga_Business_Directory.csv")

#NAICSTitle and NAICSDescr are in the wrong columns from 2016 to 2018.  2019 and 2021 are correct.
#Rename BID to BusinessID ; NAICSTitle to NAICSDescr ; NACISSecto to NAICSTitle ; 
df2017.rename(columns = {'BID':'BusinessID','NAICSTitle':'NAICSDescr','NAICSSecto':'NAICSCat'}, inplace = True)

# Create a column to hold the year
df2017['Year'] = 2017

#fix values on EmplRange that did not download properly in csv file from publisher site. Confirmed on json file from publisher.
df2017['EmplRange'] = df2017['EmplRange'].replace('1000 puls','1000+')

# Load the 2018 data set into a pandas DataFrame
df2018 = pd.read_csv("/content/drive/MyDrive/CIND 820 Capstone Project/2018_Mississauga_Business_Directory.csv")

#NAICSTitle and NAICSDescr are in the wrong columns from 2016 to 2018.  2019 and 2021 are correct.
#Rename ID1 to BusinessID ; NAICSTitle to NAICSDescr ; NAICSDescri to NAICSTitle
df2018.rename(columns = {'ID1':'BusinessID','NAICSTitle':'NAICSDescr','NACSDescri':'NAICSCat','Street_Add':'Address','Bldg_No':'BldgNo','Unit_No':'UnitNo','Modified_D':'EmplUpdate'}, inplace = True)

# Create a column to hold the year
df2018['Year'] = 2018
#correct the EmplRange category
df2018['EmplRange'] = df2018['EmplRange'].replace('1000 puls','1000+')

#eliminate the duplicate record for BusinessID 85606
df2018.drop(df2018[df2018['FID'] == 4602].index, inplace = True)

# Load the 2019 data set into a pandas DataFrame
df2019 = pd.read_csv("/content/drive/MyDrive/CIND 820 Capstone Project/2019_Mississauga_Business_Directory.csv")

#Rename ID to BusinessID ; NAICSTitle to NAICS_Title
df2019.rename(columns = {'ID':'BusinessID','NAICSTitle':'NAICSCat','Modified':'EmplUpdate'}, inplace = True)
#fix one bad NAICSCode based on what I know of the NAICS Code for this business in other years
df2019['NAICSCode'] = df2019['NAICSCode'].replace(-812910, 812910)


# Create a column to hold the year
df2019['Year'] = 2019
#correct the EmplRange category
df2019['EmplRange'] = df2019['EmplRange'].replace('1000 plus','1000+')

# Load the 2021 data set into a pandas DataFrame
df2021 = pd.read_csv("/content/drive/MyDrive/CIND 820 Capstone Project/2021_Mississauga_Business_Directory.csv")

#Rename ID to BusinessID ; NAICSTitle to NAICS_Title
df2021.rename(columns = {'ID':'BusinessID','NAICSTitle':'NAICSCat'}, inplace = True)

# Create a column to hold the year
df2021['Year'] = 2021
#correct the EmplRange category
df2021['EmplRange'] = df2021['EmplRange'].replace('1000 plus','1000+')

# Merge the 2016, 2017 and 2018 data sets on the common identifier BusinessID
#merged_df = pd.merge(df2016, df2017, on='BusinessID')
#merged_traindf = pd.merge(merged_df, df2018, on='BusinessID')

#Concatenate 2016, 2017, 2018, 2019 and 2021 data sets
merged_df1 = pd.concat([df2016, df2017], axis=0)
merged_df2 = pd.concat([merged_df1, df2018], axis=0)
merged_df3 = pd.concat([merged_df2, df2019], axis=0)
merged_complete_df = pd.concat([merged_df3, df2021], axis=0)

# Add a new unique identifier "RecordID" to the merged data set
#merged_testdf['RecordID'] = range(1, len(merged_df) + 1)
merged_complete_df['RecordID'] = merged_complete_df.BusinessID.map(hash)

#fix two bad NAICSCat based on what I know of THE NAICS Category and NAICS Code for these two business in other years
merged_complete_df['NAICSCat'] = merged_complete_df['NAICSCat'].replace('Accommodatio','Accommodation')
merged_complete_df['NAICSDescr'] = merged_complete_df['NAICSDescr'].replace('813310 - Social Advocacy Organizations','Social Advocacy Organizations')
#fix a record with an incorrect ward based on what I know about the same record in other years
merged_complete_df['Ward'] = merged_complete_df['Ward'].replace(105,10)
#fix a record with an incorrect postal code based on the postal code the business has in other years
merged_complete_df['PostalCode'] = merged_complete_df['PostalCode'].replace("v","L5G 4P3")

#Is the business new to the directory in a given year?  Create a column 'isnew' and populate with true or false.
s1 = merged_complete_df.groupby('Year')['BusinessID'].value_counts()
s2 = s1.unstack().diff().replace({0: False, np.nan: True}).stack()
merged_complete_df['isnew'] = merged_complete_df.set_index(['Year', 'BusinessID']).index.map(s2)

#Mark the last year a business existed in the directory by populating a column to hold the Closed status
merged_complete_df['Closed'] = 'No'
not_last_rows = merged_complete_df['BusinessID'].duplicated(keep='last')
merged_complete_df.loc[~not_last_rows, 'Closed'] = 'Yes'

#Remove the closed status from the 2021 records
merged_complete_df.loc[merged_complete_df['Year'] == 2021, 'Closed'] = 'No'

#Missing values in the EmplRange field account for 2646 of 78032 records or 3.39%
#Used backwards and then forwards fill na values in emplrange field as these are actual values for companies
merged_complete_df['EmplRange'] = merged_complete_df.groupby('BusinessID')['EmplRange'].ffill().bfill()

#Missing values in the NAICSCode NAICSCat field account for 145 of 78032 records or 0.185%
#change NAICSCode 1 to no value
merged_complete_df['NAICSCode'] = merged_complete_df['NAICSCode'].replace(1, np.NAN)
merged_complete_df['NAICSCode'] = merged_complete_df['NAICSCode'].replace(r'^\s*$', np.nan, regex=True)

#Used backwards and then forwards fill na values in other fields as these are actual values for companies
merged_complete_df['NAICSCode'] = merged_complete_df.groupby('BusinessID')['NAICSCode'].ffill().bfill()

merged_complete_df['NAICSDescr'] = merged_complete_df['NAICSDescr'].replace(r'^\s*$', np.nan, regex=True)
merged_complete_df['NAICSDescr'] = merged_complete_df.groupby('BusinessID')['NAICSDescr'].ffill().bfill()

merged_complete_df['NAICSCat'] = merged_complete_df['NAICSCat'].replace(r'^\s*$', np.nan, regex=True)
merged_complete_df['NAICSCat'] = merged_complete_df.groupby('BusinessID')['NAICSCat'].ffill().bfill()

# Save the merged data set to a new file
merged_complete_df.to_csv("/content/drive/MyDrive/CIND 820 Capstone Project/merged_completedata.csv", index=False)

#Create closed company data frame
closed_df = merged_complete_df
#Add a year closed column and bump up one year past last year in directory
closed_df = closed_df[closed_df['Closed'] == 'Yes'] 
closed_df['YearClosed'] = closed_df.groupby('BusinessID').Year.transform(np.roll, shift=-1)
#create an opened company data frame
opened_df = merged_complete_df
opened_df = opened_df[opened_df['isnew'] == 'Yes'] 

# Save the closed and opened data sets to new files
closed_df.to_csv("/content/drive/MyDrive/CIND 820 Capstone Project/closed_data.csv", index=False)
opened_df.to_csv("/content/drive/MyDrive/CIND 820 Capstone Project/opened_data.csv", index=False)

# Merge the 2019 and 2021 data sets on the common identifier BusinessID
merged_traintestdf = pd.merge(df2019, df2021, on='BusinessID')

# Add a new unique identifier "RecordID" to the merged data set
merged_traintestdf['RecordID'] = range(1, len(merged_traintestdf) + 1)
merged_traintestdf['RecordID'] = merged_traintestdf.BusinessID.map(hash)

# Save the merged data set to a new file
merged_traintestdf.to_csv("/content/drive/MyDrive/CIND 820 Capstone Project/merged_traintestdata.csv", index=False)



<ipython-input-73-8042712e86de>:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_df['YearClosed'] = closed_df.groupby('BusinessID').Year.transform(np.roll, shift=-1)
